In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Define the Q-network
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Hyperparameters
state_size = 5  # Example: [predicted outbreak risk, region, time, etc.]
action_size = 3  # Example: [quarantine, resource allocation, advisories]
learning_rate = 0.001
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration-exploitation tradeoff
epsilon_min = 0.01
epsilon_decay = 0.995
batch_size = 64
memory = []  # Replay memory for experience replay
memory_capacity = 10000

# Initialize the Q-network and optimizer
q_network = DQN(state_size, action_size)
optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Experience replay function
def replay(memory, batch_size):
    if len(memory) < batch_size:
        return
    
    batch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*batch)
    
    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.long)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    dones = torch.tensor(dones, dtype=torch.float32)

    # Get the current Q values
    q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    
    # Get the maximum predicted Q value for the next states
    next_q_values = q_network(next_states).max(1)[0]
    target_q_values = rewards + (gamma * next_q_values * (1 - dones))
    
    loss = criterion(q_values, target_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Step function to take action and observe the reward
def step(state, action):
    # Placeholder function to simulate environment interaction
    # This function should return next_state, reward, done (True if episode ends)
    pass

# Train the DQN with experience replay
for episode in range(1000):
    state = np.random.rand(state_size)  # Initialize the state
    
    for t in range(200):  # Limit the episode to 200 steps
        if random.random() < epsilon:
            action = random.choice(range(action_size))  # Exploration
        else:
            action = q_network(torch.tensor(state, dtype=torch.float32)).argmax().item()  # Exploitation

        # Take action and observe the next state and reward
        next_state, reward, done = step(state, action)
        
        # Store the experience in memory
        memory.append((state, action, reward, next_state, done))
        if len(memory) > memory_capacity:
            memory.pop(0)
        
        # Perform experience replay
        replay(memory, batch_size)
        
        state = next_state
        
        if done:
            break
    
    # Decay epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f"Episode {episode + 1}, Epsilon: {epsilon:.3f}")
